# RNA MULTICAPA DE 3 CAPAS
 - capa de entrada
 - 1 capa oculta
 - capa de salida)

## importar librerias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

## funcion de activacion

In [ ]:
def func_activacion(n):
  return 1/(1+math.exp(-n))   

In [ ]:
func_activacion_vect = np.vectorize(func_activacion)

## RNA

In [ ]:
def neurona(pesos,entrada, bias):
  prod_punto = np.dot(pesos,entrada)+bias # z = W.X + b
  return func_activacion_vect(prod_punto) # f(z)

## RNA Multicapa

In [ ]:
def red_neuronal_multicapa(input, pesosc1, biasc1, pesoscs, biascs):
  # Capa entrada 
  entrada_x = input
  # Capa oculta 1
  salida_c1 = neurona(pesosc1, entrada_x, biasc1)
  # Capa de SALIDA
  output_cs = neurona(pesoscs, salida_c1, biascs)
  # Retornar resultados  
  return salida_c1, output_cs

## derivada de la funcion

In [ ]:
def derivada_f(salida_y): # derivada de la funcion sigmoide
  return salida_y*(1-salida_y)

In [ ]:
def adjust_output(output_wish,size,ExistZero):
  SD = np.zeros(size) # vector con el nro de neuronas 
  if ExistZero == True:
    SD[output_wish] = 1
  else:
    SD[output_wish - 1] = 1
  return SD

## backpropagation

In [ ]:
def propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error=0.005,epochs=1000):
  flag = True if 0 in salidas_deseadas.unique() else False
  # longitud de la entrada(caracteristicas)
  n_entradas = entradas.shape[1]

  # Capa oculta 1
  pesosc1 = 2*np.random.rand(neuronas_c1,n_entradas) -1   # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
  biasc1 = 2*np.random.rand(neuronas_c1)-1
  # Capa de salida
  pesoscs = 2*np.random.rand(neuronas_cs,neuronas_c1) - 1 # filas: nro neuronas x capa, col = nro inputs que recibe la neurona
  biascs = 2*np.random.rand(neuronas_cs) - 1

  # Iterar el algoritmo(nro de epocas)
  for epoch in range(0,epochs):
    # Evaluar los patrones(para c/entrada y salida) respectivamente
    for entrada_i,salida_deseada_i in zip(entradas,salidas_deseadas):
      # Predecir usando la red neuronal
      salida_c1, output_cs = red_neuronal_multicapa(entrada_i, pesosc1, biasc1, pesoscs, biascs)

      # Calcular los errores producidos en cada capa de forma invertida(retropropagacion) y actualizar pesos
      # ----------------------------------------------------------------------------------------------------
      salida_deseada = adjust_output(salida_deseada_i,neuronas_cs,flag)
      # Calcular Δ para la CAPA de SALIDA(cs): 
      # Δ Salida_Y = f'(Salida_Y) * (salidaY_Deseada - Salida_Y) 
      # F' de la func sigmoide: f'(Salida_Y) = Salida_Y * (1 - Salida_Y)
      delta_cs = derivada_f(output_cs)*(salida_deseada - output_cs)
      peso_anterior_cs = pesoscs
      
      # Actualizar pesos que llegan a la capa de SALIDA
      # W = W_ingresan_a_las_ncs + alfa*Salida_y(capa anterior)*Δ_cs
      pesoscs = pesoscs + alfa * delta_cs.reshape(1,-1).T * salida_c1.reshape(1,-1) 
      biascs = biascs + alfa * delta_cs*1   # b = b + alfa*Δ

      # Calcular Δ para la CAPA OCULTA 1 (pesosc1):      
      # Δ Salida_Y = f'(Salida_Y) * (W_entran_c1 * ΔSalida_CS)
      delta_c1 = derivada_f(salida_c1) * np.dot(peso_anterior_cs.T, delta_cs) 
      
      # Actualizar pesos  que entran a las neuronas de la capa oculta 1
      # W = W_ingresan_a_las_ncs + alfa*Salida_y(capa anterior)*Δ_c1
      pesosc1 = pesosc1 + alfa * (delta_c1.reshape(1,-1)*entrada_i.reshape(1,-1).T).T 
      biasc1 = biasc1 + alfa * delta_c1 * 1  # b = b + alfa*Δ_c1
      
  return pesosc1,biasc1,pesoscs,biascs 

## fit RNA

In [ ]:
def fit_rna(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs):
  pesosc1,biasc1,pesoscs,biascs = propagacion_hacia_atras(entradas,salidas_deseadas,alfa,neuronas_c1,neuronas_cs,error,epochs)
  return pesosc1,biasc1,pesoscs,biascs


## predict RNA

In [ ]:
def predict_rna(x_test, valores_obtenidos):
  pesosc1,biasc1,pesoscs,biascs = valores_obtenidos[0],valores_obtenidos[1],valores_obtenidos[2],valores_obtenidos[3]
  y_predict_list = []
  for entrada in x_test:
    salida_c1, output_cs = red_neuronal_multicapa(entrada, pesosc1,biasc1,pesoscs,biascs)
    y_predict_list.append(output_cs)
  return y_predict_list

# DETECCION DE DIGITOS

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix

# digitos

In [ ]:
df = pd.read_csv("digito8x8.csv")
df.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P56,P57,P58,P59,P60,P61,P62,P63,P64,digito
0,0,1,6,15,12,1,0,0,0,7,...,0,0,0,6,14,7,1,0,0,0
1,0,0,10,16,6,0,0,0,0,7,...,0,0,0,10,16,15,3,0,0,0
2,0,0,8,15,16,13,0,0,0,1,...,0,0,0,9,14,0,0,0,0,7
3,0,0,0,3,11,16,0,0,0,0,...,0,0,0,0,1,15,2,0,0,4
4,0,0,5,14,4,0,0,0,0,0,...,0,0,0,4,12,14,7,0,0,6


In [ ]:
df['digito'].unique()

array([0, 7, 4, 6, 2, 5, 8, 1, 9, 3])

In [ ]:
# =========== NORMALIZANDO ================== 
X = df.drop("digito", axis=1)
Y = df["digito"]

scaler = preprocessing.StandardScaler()
scaler.fit(X)
# =========== TRANSFORMANDO ================= 
X = scaler.transform(X)

In [ ]:
# =========== SEPARACION DE DATOS =========== 
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
# =========== FIT ===========================  
pesosc1,biasc1,pesoscs,biascs = fit_rna(X_train,y_train,0.1,128,10,0.005,100)

In [ ]:
# =========== PREDICT ======================= 
pesos_bias_optimos = [pesosc1,biasc1,pesoscs,biascs]   # recuperar los pesos y bias optimos
y_pred_ = predict_rna(X_test, pesos_bias_optimos) # predecir

In [ ]:
# =========== ajustar salida ================ 
def ajuste(valores_pred):
  result_pred = []
  for vector in valores_pred:
    indice = np.argmax(vector)
    result_pred.append(indice)
  y_pred = pd.Series(result_pred)
  return result_pred

In [ ]:
y_pred = ajuste(y_pred_) # ajustar valores de salida

In [ ]:
# =========== METRICAS DE EVALUACION ========= 
print(classification_report(y_test,y_pred ))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       108
           1       0.96      0.97      0.97       102
           2       0.99      0.98      0.99       107
           3       0.97      0.97      0.97       118
           4       0.97      0.99      0.98       117
           5       0.96      0.98      0.97        97
           6       0.97      0.98      0.97       123
           7       1.00      1.00      1.00       124
           8       0.97      0.96      0.97       105
           9       0.97      0.93      0.95       123

    accuracy                           0.98      1124
   macro avg       0.98      0.98      0.98      1124
weighted avg       0.98      0.98      0.98      1124



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score, f1_score

valores_pred = predict_rna(X_train, pesos_bias_optimos)
y_train_pred = pd.Series(ajuste(valores_pred))


# verificando aprendizaje
print("Aprendizaje - Entrenamiento")
print(balanced_accuracy_score(y_train, y_train_pred))
print(f1_score(y_train, y_train_pred, average="weighted"))

print('\n')
#verificando generalización
print("Prueba - Generalización")
print(balanced_accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average="weighted"))

Aprendizaje - Entrenamiento
0.9980004402012904
0.9979987231825675


Prueba - Generalización
0.9761306643772105
0.9759393615568811


## prediccion

In [ ]:
entrada = [0,  0,  6, 15, 11,  1,  0,  0,  0,  7, 15,  6,  6, 10,  0,  0,  0,
            8, 16,  2,  0, 10,  2,  0,  0,  5, 16,  3,  0,  5,  7,  0,  0,  7,
          13,  3,  0,  8,  7,  0,  0,  4, 12,  0,  1, 13,  5,  0,  0,  0, 14,
            9, 15,  9,  0,  0,  0,  0,  6, 14,  7,  1,  0,  0]

In [ ]:
entrada_df = np.array([entrada]) # convertir a array de numpy y lista de listas
# NORMALIZANDO LOS DATOS DE ENTRADA
entrada_df = scaler.transform(entrada_df)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
entrada_df 

array([[ 0.        , -0.343381  ,  0.12986348,  0.74831744, -0.12959939,
        -0.81585144, -0.41097194, -0.13206188, -0.03542401,  1.62296632,
         0.82585557, -1.44911936, -0.94280989,  0.29178646, -0.53252006,
        -0.1466391 , -0.04069176,  1.5466172 ,  1.08427078, -0.8190465 ,
        -1.16349493,  0.32404697,  0.01042016, -0.11335335, -0.03269184,
         0.84415735,  1.10740032, -1.02318838, -1.56286359, -0.46403448,
         1.2807267 , -0.04777844, -0.02984078,  1.47241212,  0.85010099,
        -0.99520444, -1.74442775, -0.17931693,  1.16744553,  0.        ,
        -0.07872062,  0.86007938,  0.82966502, -1.11476112, -1.08631271,
         0.77341948,  0.34592509, -0.08987191, -0.0576035 , -0.40589805,
         1.10346414, -0.1498682 ,  1.0064432 , -0.019331  , -0.76183046,
        -0.19789895, -0.01886792, -0.30310217,  0.04770289,  0.46244502,
        -0.9190364 , -0.98379588, -0.51724355, -0.1791362 ]])

In [ ]:
result = predict_rna(entrada_df, pesos_bias_optimos) 

In [ ]:
result = predict_rna(entrada_df, pesos_bias_optimos) 
np.argmax(result)

0